In [ ]:
# Environment setup

!sudo apt-get install -y pciutils
! sudo apt-get install zstd -y
!curl -fsSL https://ollama.ai/install.sh | sh

! pip install ollama
! pip install langchain-community
! pip install langchain-ollama
! pip install llama-index-llms-ollama
! pip install llama-index
! pip install langchain-community
! pip install langchain-text-splitters
! pip install langchain-huggingface
! pip install chromadb
! pip install langchain-chroma
! pip install langchain-ollama
! pip install sentence-transformers
! pip install unstructured
! pip install unstructured[pdf] nltk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
! ollama serve > server.log 2>&1 &
! sleep 60s

In [ ]:
# !ollama pull qwen3-embedding
! ollama pull qwen3:1.7b

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

In [ ]:
def main():
    print("Hello from langchain-course!")
    information = """
    Elon Reeve Musk FRS (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman, known for his leadership of Tesla, SpaceX, X (formerly Twitter), and the Department of Government Efficiency (DOGE). Musk has been the wealthiest person in the world since 2021; as of May 2025, Forbes estimates his net worth to be US$424.7 billion.

Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He received bachelor's degrees from the University of Pennsylvania in 1997 before moving to California, United States, to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also became an American citizen.

In 2002, Musk founded the space technology company SpaceX, becoming its CEO and chief engineer; the company has since led innovations in reusable rockets and commercial spaceflight. Musk joined the automaker Tesla as an early investor in 2004 and became its CEO and product architect in 2008; it has since become a leader in electric vehicles. In 2015, he co-founded OpenAI to advance artificial intelligence (AI) research but later left; growing discontent with the organization's direction and their leadership in the AI boom in the 2020s led him to establish xAI. In 2022, he acquired the social network Twitter, implementing significant changes and rebranding it as X in 2023. His other businesses include the neurotechnology company Neuralink, which he co-founded in 2016, and the tunneling company the Boring Company, which he founded in 2017.

Musk was the largest donor in the 2024 U.S. presidential election, and is a supporter of global far-right figures, causes, and political parties. In early 2025, he served as senior advisor to United States president Donald Trump and as the de facto head of DOGE. After a public feud with Trump, Musk left the Trump administration and announced he was creating his own political party, the America Party.

Musk's political activities, views, and statements have made him a polarizing figure, especially following the COVID-19 pandemic. He has been criticized for making unscientific and misleading statements, including COVID-19 misinformation and promoting conspiracy theories, and affirming antisemitic, racist, and transphobic comments. His acquisition of Twitter was controversial due to a subsequent increase in hate speech and the spread of misinformation on the service. His role in the second Trump administration attracted public backlash, particularly in response to DOGE.
    """

    summary_template = """
    given the information \n\n {information} \n\n about a person I want you to create:\n\n
    1. A short summary
    2. two interesting facts about them
    """

    summary_prompt_template = PromptTemplate(
        input_variables=["information"], template=summary_template
    )

    print(summary_prompt_template.format(information=information))


    llm = ChatOllama(temperature=0, model="qwen3:1.7b")
    chain = summary_prompt_template | llm

    response = chain.invoke(input={"information": information})
    print(response.content)

In [ ]:
main()

Hello from langchain-course!

    given the information 

 
    Elon Reeve Musk FRS (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman, known for his leadership of Tesla, SpaceX, X (formerly Twitter), and the Department of Government Efficiency (DOGE). Musk has been the wealthiest person in the world since 2021; as of May 2025, Forbes estimates his net worth to be US$424.7 billion.

Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He received bachelor's degrees from the University of Pennsylvania in 1997 before moving to California, United States, to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also became an American citizen.

In 2002, Musk founded the space technology company SpaceX, becoming its CEO and chief engineer; the company has since led innovat

In [ ]:
# Creating an agent

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from pydantic import BaseModel, Field

def search(query):
  """
  Tool that searches over internet
  Args:
    query: The query to search for
  Returns:
    The search results
  """
  print(f"Searching for {query}")
  struct = {}
  struct['answer'] = "Tokyo is sunny today"
  struct['sources'] = ["source-1"]
  return struct

prompt = (
    "You MUST use the search tool.\n"
    "The tool returns a JSON object with keys: answer, sources.\n"
    "Do not use your own knowledge and respond with I dont know if the relevant information isn't returned"
)

llm = ChatOllama(temperature=0, model="qwen3:1.7b")
tools = [search]
agent = create_agent(model=llm, tools=tools, response_format=AgentResponse, system_prompt=prompt)
print(agent)
result = agent.invoke({"messages": HumanMessage(content="What is the weather in India?")})


Searching for weather in India


In [ ]:
print(result['messages'])

NameError: name 'result' is not defined

In [ ]:
class Source(BaseModel):
  """
  Schema for a source used by an agent
  """
  url: str = Field(description="The URL of the source")

class AgentResponse(BaseModel):
  """ Schema for agent response with answer and sources """
  answer: str = Field(description="The answer to the question")
  sources: list[Source] = Field(default_factory=list, description="The sources used to answer the question")


ModuleNotFoundError: No module named 'langchain.chains'